In [1]:
# importing libraries
import pandas as pd
import os
from datetime import datetime
import shutil
import time

### Global variables below

In [2]:
# Create a file path to the two directories where the input file is stored
aris_input_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\1 - Processes\\1.1 - ARIS GCRED Monthly File\\Inputs\\1.1.1 - ARIS'
aris_archive_folder = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\1 - Processes\\1.1 - ARIS GCRED Monthly File\\Inputs\\1.1.1 - ARIS\\Archive'
gcred_input_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\1 - Processes\\1.1 - ARIS GCRED Monthly File\\Inputs\\1.1.2 - GCRED'
gcred_archive_folder =r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\1 - Processes\\1.1 - ARIS GCRED Monthly File\\Inputs\\1.1.2 - GCRED\\Archive'
output_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\1 - Processes\\1.1 - ARIS GCRED Monthly File\\Outputs'

# Outlining columns that need to be formatted in each sheet in both Excel Workbooks 

# First outlining the calculations that will be done between the two datasets
div_by_100 = 10**2
div_by_1k = 10**3
div_by_10k = 10**4
div_by_1billion = 10**9
div_by_10_billion = 10**10

# Mapping the columns to each calculation
data_distribution_columns_to_format = {
    'Distribution Rate Per Share': div_by_10_billion,
    'Fund Expense Rate': div_by_10_billion,
    'Fund Expense Amount': div_by_100,
    'Shares Reinvested': div_by_10k,
    'Share Balance After Distribution': div_by_10k,
    'Issued Shares/Units': div_by_10k,
    'Distribution Amount': div_by_100,
    'Subaccounting Fee Amount': div_by_100,
    'Other Fees Amount': div_by_100,
    'Foreign Tax Withholding Rate': div_by_1k,
    'Foreign Tax Withholding Amount': div_by_100,
    'US Withholding Amount': div_by_100
}

data_financial_activity_columns_to_format = { 
    'Unit Price Amount': div_by_10k, 
    'Gross Transaction Amount': div_by_100,
    'Unit Transaction Count': div_by_10k, 
    'Percent Sales Charge': div_by_10_billion, 
    'Dealer Commission Amount': div_by_100,
    'Underwriter Commission Amount': div_by_100
}

data_account_position_columns_to_format = {
    'Total Units': div_by_10k, 
    'Unissued Units': div_by_10k, 
    'Issued Units': div_by_10k,
    'Unit Price Amount': div_by_10k,
    'Accrued Dividend Amount': div_by_100,
    'Escrow Shares Count': div_by_10k,
    'Amount Invested': div_by_100, 
    'Collected Units': div_by_10k
}

data_price_refresher_columns_to_format = {
    'Unit Price Amount': div_by_10k,
    'Public Offering Price': div_by_10k,
    'Daily Dividend Rate': div_by_1billion
}

# Create a mapping of dataframes to formatting rule
dataframes_to_format = {
    'Distribution': data_distribution_columns_to_format,
    'Financial Activity': data_financial_activity_columns_to_format,
    'Account Position': data_account_position_columns_to_format,
    'Non Financial Activity': None,
    'New Account Activity': None,
    'Price Refresher': data_price_refresher_columns_to_format   
}

In [3]:
# Establishing date information
date_formatted = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
year = date_formatted[:4]
month = date_formatted[5:7]
day = date_formatted[8:10]
date_short = year + '-' + month + '-' + day

### ARIS variables below

In [4]:
# Setting the directory then importing ARIS data with selected sheets
os.chdir(aris_input_path)
aris_input_file_name = 'Copy of ARIS Monthly File.xlsx'
aris_xlsx = pd.ExcelFile(aris_input_file_name)
aris_sheets = aris_xlsx.sheet_names
aris_data_dictionary = pd.read_excel(aris_input_file_name, sheet_name = aris_sheets)

# Unpacking the ARIS dictionary into a dataframe for each sheet
aris_data_distribution = aris_data_dictionary['Distribution']
aris_data_financial_activity = aris_data_dictionary['Financial Activity']
aris_data_account_position = aris_data_dictionary['Account Position']
aris_data_price_refresher = aris_data_dictionary['Price Refresher']
aris_data_non_financial_activity = aris_data_dictionary['Non Financial Activity']
aris_data_new_account_activity = aris_data_dictionary['New Account Activity']

In [5]:
# Building out a function to apply formatting to all columns in the dataset selected
def format_columns_for_selected_data(data, cols_2_format):
    # breaking out the dictionary for cols_2_format so the loop uses the dictionary properly for the transformation
    for col, division_factor in cols_2_format.items():
        # apply division to the column if it is in the list of columns to format and in the dataset
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce') / division_factor
    
    return data

In [6]:
# Apply transformations to each dataset in sequence outlined above
for sheet_name, format_rules in dataframes_to_format.items():
    if format_rules:
        # transformation rules outlined here
        aris_data_dictionary[sheet_name] = format_columns_for_selected_data(
            aris_data_dictionary[sheet_name], 
            format_rules
        )

In [7]:
# Establishing the output path
output_file = os.path.join(output_path, f'ARIS Monthly File {date_short}.xlsx')

# Writing each dataset to its own sheet in the output file
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name in dataframes_to_format.keys():
        aris_data_dictionary[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)



In [8]:
os.makedirs(aris_archive_folder, exist_ok=True)

aris_original_filename = os.path.basename(aris_input_file_name)
name, ext = os.path.splitext(aris_original_filename)

# Create the new filename with date_formatted appended
archive_file_name = f"{name} {date_formatted}{ext}"

# Create the full destination path
destination = os.path.join(aris_archive_folder, archive_file_name)  # FIXED: Added archive_file_name back

try:
    time.sleep(1)  # Give time for any file handles to release
    shutil.move(src=aris_input_file_name, dst=destination)
    print(f"File successfully moved to: {destination}")
except PermissionError as e:
    print(f"Warning: Could not move file - {e}")
    print("The file may be open in Excel or another program.")
# REMOVED: The duplicate shutil.move() line that was here


The file may be open in Excel or another program.


In [9]:
# deleting some variables before GCRED for performance
del aris_data_account_position
del aris_data_dictionary
del aris_data_distribution
del aris_data_financial_activity
del aris_data_new_account_activity
del aris_data_non_financial_activity
del aris_data_price_refresher
del aris_input_path
del aris_sheets
del aris_xlsx
del aris_input_file_name

In [10]:
# Setting the directory then importing ARIS data with selected sheets
os.chdir(gcred_input_path)
gcred_input_file_name = 'GCRED Monthly File.xlsx'
gcred_xlsx = pd.ExcelFile(gcred_input_file_name)
gcred_sheets = gcred_xlsx.sheet_names
gcred_data_dictionary = pd.read_excel(gcred_input_file_name, sheet_name = gcred_sheets)

# Unpacking the ARIS dictionary into a dataframe for each sheet
gcred_data_distribution = gcred_data_dictionary['Distribution']
gcred_data_financial_activity = gcred_data_dictionary['Financial Activity']
gcred_data_account_position = gcred_data_dictionary['Account Position']
gcred_data_price_refresher = gcred_data_dictionary['Price Refresher']
gcred_data_non_financial_activity = gcred_data_dictionary['Non Financial Activity']
gcred_data_new_account_activity = gcred_data_dictionary['New Account Activity']

In [11]:
# Same structure as for loop above, review that cell for additional details
for sheet_name, format_rules in dataframes_to_format.items():
    if format_rules:  
        gcred_data_dictionary[sheet_name] = format_columns_for_selected_data(
            gcred_data_dictionary[sheet_name], 
            format_rules
        )

In [12]:
# Establishing the output path
output_file = os.path.join(output_path, f'GCRED Monthly File {date_short}.xlsx')

# Creating a sheet for each dataframe
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name in dataframes_to_format.keys():
        gcred_data_dictionary[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

In [13]:
date_formatted = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
year = date_formatted[:4]
month = date_formatted[5:7]
day = date_formatted[8:10]
date_short = year + '/' + month + '/' + day
date_short

'2025/12/10'